In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "growing_degree_days_gt_4"

In [4]:
#Variable 
variable85="rcp85_"+var+"_monthly"
variable45="rcp45_"+var+"_monthly"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-cl

In [5]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.month[0:12]
# Monthly example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * time                      (time) datetime64[ns] 1981-01-01 ... 2100-01-01
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset

## Loop to create all 12 dataframes

In [7]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df45=0
    df453 = df452.loc[df452["realization"] == 0].dropna()
    df452=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    dfp=0
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"], 2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"], 2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"], 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45C=0
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    Region1i45=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)         
    df45f.columns = [month + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF 

In [8]:
df45list


[                      january_hist_rcp45_p10  january_t2050_rcp45_p10  \
 lat       lon                                                           
 40.041039 -89.045212                    0.04                     2.61   
           -88.961884                    0.04                     2.59   
           -88.878548                    0.05                     2.55   
           -88.795219                    0.04                     2.48   
           -88.711891                    0.05                     2.48   
 ...                                      ...                      ...   
 66.623306 -61.629639                    0.00                     0.00   
           -61.546310                    0.00                     0.00   
           -61.462978                    0.00                     0.00   
           -61.379650                    0.00                     0.00   
           -61.296322                    0.00                     0.00   
 
                       january_t2080

In [9]:
def merge_multiple_dfs(df_list, on):
    df_result = df_list[0]
    for df in df_list[1:]:
        df_result = df_result.merge(df, on=on)
    return df_result

In [10]:
mkey = ["lat", "lon"]

In [11]:
dfse45 = merge_multiple_dfs(df45list[0:12], on=mkey)

In [12]:
dfse452 = dfse45.reset_index()
dfse45=0

In [13]:
dfse452.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [14]:
dfse452 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [15]:
dfse453 = dfse452.drop(columns=["january_hist_rcp45_p10","january_hist_rcp45_p50", "january_hist_rcp45_p90", 
                               "february_hist_rcp45_p10", "february_hist_rcp45_p50", "february_hist_rcp45_p90", 
                               "march_hist_rcp45_p10","march_hist_rcp45_p50", "march_hist_rcp45_p90", 
                               "april_hist_rcp45_p10","april_hist_rcp45_p50", "april_hist_rcp45_p90",
                              "may_hist_rcp45_p10","may_hist_rcp45_p50", "may_hist_rcp45_p90", 
                               "june_hist_rcp45_p10", "june_hist_rcp45_p50", "june_hist_rcp45_p90", 
                               "july_hist_rcp45_p10","july_hist_rcp45_p50", "july_hist_rcp45_p90", 
                               "august_hist_rcp45_p10","august_hist_rcp45_p50", "august_hist_rcp45_p90",
                              "september_hist_rcp45_p10","september_hist_rcp45_p50", "september_hist_rcp45_p90", 
                               "october_hist_rcp45_p10", "october_hist_rcp45_p50", "october_hist_rcp45_p90", 
                               "november_hist_rcp45_p10","november_hist_rcp45_p50", "november_hist_rcp45_p90", 
                               "december_hist_rcp45_p10","december_hist_rcp45_p50", "december_hist_rcp45_p90"])
dfse453


,lat,lon,january_t2050_rcp45_p10,january_t2080_rcp45_p10,january_t2050_rcp45_p50,january_t2080_rcp45_p50,january_t2050_rcp45_p90,january_t2080_rcp45_p90,february_t2050_rcp45_p10,february_t2080_rcp45_p10,...,november_t2050_rcp45_p50,november_t2080_rcp45_p50,november_t2050_rcp45_p90,november_t2080_rcp45_p90,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,2.61,2.14,13.14,16.82,38.80,42.60,8.20,13.17,...,140.72,152.36,204.98,218.14,8.74,11.54,30.57,34.57,71.92,80.06
1,40.041039,-88.961884,2.59,2.11,12.97,16.69,38.51,42.25,8.07,13.08,...,140.49,152.00,204.58,217.79,8.68,11.43,30.42,34.18,71.65,79.53
2,40.041039,-88.878548,2.55,2.09,12.71,16.48,38.01,41.76,7.96,13.00,...,140.28,151.62,204.17,217.41,8.65,11.35,30.30,33.92,71.55,79.23
3,40.041039,-88.795219,2.48,2.02,12.31,16.19,37.39,41.05,7.77,12.78,...,139.53,150.89,203.10,216.44,8.54,11.29,30.11,33.56,71.12,78.81
4,40.041039,-88.711891,2.48,2.05,12.23,16.24,37.15,40.84,7.73,12.77,...,139.90,151.41,203.15,216.76,8.66,11.33,30.32,33.63,71.36,79.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00


In [16]:
lisdfmoh45 = []
for month in monthlist: 
    dfmoh = dfse452[["lat", "lon", month+"_hist_rcp45_p50"]]
    print (month)
    lisdfmoh45.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [17]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
# Monthly example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * time                      (time) datetime64[ns] 1981-01-01 ... 2100-01-01
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset

## Loop to create all 12 dataframes

In [18]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df85=0
    df853 = df852.loc[df852["realization"] == 0].dropna()
    df852=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    dfp852=0
    df85C[var+"_p10"] = round(df85C[var+"_p10"], 2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"], 2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"], 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    Region1i85=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)
    df85f.columns = [month + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF added to the list
Dataset number:  5
Percentiles Obteined
Converted to DF
S

In [19]:
df85list


[                      january_hist_rcp85_p10  january_t2050_rcp85_p10  \
 lat       lon                                                           
 40.041039 -89.045212                    0.06                     3.32   
           -88.961884                    0.06                     3.27   
           -88.878548                    0.05                     3.19   
           -88.795219                    0.06                     3.08   
           -88.711891                    0.06                     3.03   
 ...                                      ...                      ...   
 66.623306 -61.629639                    0.00                     0.00   
           -61.546310                    0.00                     0.00   
           -61.462978                    0.00                     0.00   
           -61.379650                    0.00                     0.00   
           -61.296322                    0.00                     0.00   
 
                       january_t2080

In [20]:
dfse85 = merge_multiple_dfs(df85list[0:12], on=mkey).reset_index()

In [21]:
dfse85

,lat,lon,january_hist_rcp85_p10,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_hist_rcp85_p50,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_hist_rcp85_p90,january_t2050_rcp85_p90,...,november_t2080_rcp85_p90,december_hist_rcp85_p10,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_hist_rcp85_p50,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_hist_rcp85_p90,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,0.06,3.32,11.53,3.86,17.42,34.08,19.61,46.40,...,277.79,1.89,14.54,27.47,16.15,38.25,61.34,45.55,88.72,131.65
1,40.041039,-88.961884,0.06,3.27,11.42,3.79,17.31,33.80,19.29,46.02,...,277.35,1.90,14.40,27.31,16.07,38.09,61.07,45.32,88.07,131.25
2,40.041039,-88.878548,0.05,3.19,11.32,3.72,17.02,33.47,18.93,45.51,...,276.95,1.92,14.35,27.23,16.12,37.96,60.86,45.29,87.71,131.04
3,40.041039,-88.795219,0.06,3.08,11.12,3.53,16.58,32.85,18.41,44.62,...,275.93,1.89,14.23,27.08,15.96,37.66,60.43,45.04,87.08,130.47
4,40.041039,-88.711891,0.06,3.03,11.11,3.53,16.50,32.83,18.17,44.36,...,276.15,1.92,14.32,27.24,16.14,37.86,60.69,45.32,87.35,130.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.81
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.63
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.64
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.58


In [22]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [23]:
dfse85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [24]:
dfse852 = dfse85.drop(columns=["january_hist_rcp85_p10","january_hist_rcp85_p50", "january_hist_rcp85_p90", 
                               "february_hist_rcp85_p10", "february_hist_rcp85_p50", "february_hist_rcp85_p90", 
                               "march_hist_rcp85_p10","march_hist_rcp85_p50", "march_hist_rcp85_p90", 
                               "april_hist_rcp85_p10","april_hist_rcp85_p50", "april_hist_rcp85_p90",
                              "may_hist_rcp85_p10","may_hist_rcp85_p50", "may_hist_rcp85_p90", 
                               "june_hist_rcp85_p10", "june_hist_rcp85_p50", "june_hist_rcp85_p90", 
                               "july_hist_rcp85_p10","july_hist_rcp85_p50", "july_hist_rcp85_p90", 
                               "august_hist_rcp85_p10","august_hist_rcp85_p50", "august_hist_rcp85_p90",
                              "september_hist_rcp85_p10","september_hist_rcp85_p50", "september_hist_rcp85_p90", 
                               "october_hist_rcp85_p10", "october_hist_rcp85_p50", "october_hist_rcp85_p90", 
                               "november_hist_rcp85_p10","november_hist_rcp85_p50", "november_hist_rcp85_p90", 
                               "december_hist_rcp85_p10","december_hist_rcp85_p50", "december_hist_rcp85_p90"])
dfse852


,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2050_rcp85_p50,november_t2080_rcp85_p50,november_t2050_rcp85_p90,november_t2080_rcp85_p90,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,3.32,11.53,17.42,34.08,46.40,84.99,9.40,25.54,...,156.26,213.33,220.93,277.79,14.54,27.47,38.25,61.34,88.72,131.65
1,40.041039,-88.961884,3.27,11.42,17.31,33.80,46.02,84.50,9.23,25.32,...,155.99,212.88,220.33,277.35,14.40,27.31,38.09,61.07,88.07,131.25
2,40.041039,-88.878548,3.19,11.32,17.02,33.47,45.51,83.86,9.12,25.18,...,155.74,212.48,219.82,276.95,14.35,27.23,37.96,60.86,87.71,131.04
3,40.041039,-88.795219,3.08,11.12,16.58,32.85,44.62,82.89,8.90,24.78,...,155.03,211.62,218.68,275.93,14.23,27.08,37.66,60.43,87.08,130.47
4,40.041039,-88.711891,3.03,11.11,16.50,32.83,44.36,82.75,8.82,24.77,...,155.55,212.10,218.79,276.15,14.32,27.24,37.86,60.69,87.35,130.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,1.65,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.81
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,1.32,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.63
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,1.33,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.64
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,1.23,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.58


In [25]:
lisdfmoh85 = []
for month in monthlist: 
    dfmoh = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    print (month)
    lisdfmoh85.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


In [26]:
lisdfmoh85

[             lat        lon  january_hist_rcp85_p50
 0      40.041039 -89.045212                    3.86
 1      40.041039 -88.961884                    3.79
 2      40.041039 -88.878548                    3.72
 3      40.041039 -88.795219                    3.53
 4      40.041039 -88.711891                    3.53
 ...          ...        ...                     ...
 76201  66.623306 -61.629639                    0.00
 76202  66.623306 -61.546310                    0.00
 76203  66.623306 -61.462978                    0.00
 76204  66.623306 -61.379650                    0.00
 76205  66.623306 -61.296322                    0.00
 
 [76206 rows x 3 columns],              lat        lon  february_hist_rcp85_p50
 0      40.041039 -89.045212                    17.09
 1      40.041039 -88.961884                    16.94
 2      40.041039 -88.878548                    16.82
 3      40.041039 -88.795219                    16.49
 4      40.041039 -88.711891                    16.38
 ...        

# Merge df45 and 85

# Extract hist

## Getting historic mean for each month

In [27]:
### Function
def mean_month(month, dfse45, dfse85):
    print(month)
    dfw45 = dfse45[["lat", "lon", month+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfw45=0
    dfw85=0
    dfsew[month+"_hist_p50"] = round((dfsew[month+"_hist_rcp45_p50"]+ dfsew[month+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dfsew=0
    dftgwh3 = dftgwh2[["lat", "lon", month+"_hist_p50"]]
    dftgwh2=0
    return (dftgwh3)


### All months historic mean

In [28]:
listdfmo = []
i=0
for month in monthlist:
    print(month)
    dfm = mean_month(month, lisdfmoh45[i], lisdfmoh85[i])
    i = i+1
    listdfmo.append(dfm)

january
january
february
february
march
march
april
april
may
may
june
june
july
july
august
august
september
september
october
october
november
november
december
december


# Merge means with the 45 and 85 DF

In [29]:
dftgseall = dfse852.merge(dfse452, on=mkey)
i=0
for df in listdfmo:
    dftgseall2 = dftgseall.merge(listdfmo[i], on=mkey)
    i = i +1


In [30]:
dftgseall

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,3.32,11.53,17.42,34.08,46.40,84.99,9.40,25.54,...,218.14,2.10,8.74,11.54,16.88,30.57,34.57,48.42,71.92,80.06
1,40.041039,-88.961884,3.27,11.42,17.31,33.80,46.02,84.50,9.23,25.32,...,217.79,2.10,8.68,11.43,16.82,30.42,34.18,48.17,71.65,79.53
2,40.041039,-88.878548,3.19,11.32,17.02,33.47,45.51,83.86,9.12,25.18,...,217.41,2.11,8.65,11.35,16.77,30.30,33.92,48.15,71.55,79.23
3,40.041039,-88.795219,3.08,11.12,16.58,32.85,44.62,82.89,8.90,24.78,...,216.44,2.05,8.54,11.29,16.64,30.11,33.56,47.83,71.12,78.81
4,40.041039,-88.711891,3.03,11.11,16.50,32.83,44.36,82.75,8.82,24.77,...,216.76,2.10,8.66,11.33,16.75,30.32,33.63,48.13,71.36,79.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,1.65,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,1.32,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,1.33,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,1.23,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00


In [31]:
dfmop50 = merge_multiple_dfs(listdfmo[0:12], on=mkey)

In [32]:
dfmop50

,lat,lon,january_hist_p50,february_hist_p50,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,4.39,16.66,82.00,233.76,416.28,555.58,634.25,600.83,465.64,272.86,102.54,16.52
1,40.041039,-88.961884,4.31,16.49,81.54,232.83,415.22,554.79,633.64,600.88,466.02,272.58,102.20,16.44
2,40.041039,-88.878548,4.20,16.37,80.86,232.00,414.92,554.72,633.43,601.17,466.56,272.31,101.78,16.44
3,40.041039,-88.795219,4.03,16.04,79.46,230.67,414.77,555.10,633.12,601.10,466.22,271.38,100.94,16.30
4,40.041039,-88.711891,4.01,15.95,79.45,230.60,415.72,556.49,633.40,601.68,467.61,271.97,101.15,16.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,10.03,67.27,52.31,1.74,0.00,0.00,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,7.90,59.52,45.55,1.08,0.00,0.00,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,9.07,64.34,49.92,1.50,0.00,0.00,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,9.41,65.80,51.24,1.66,0.00,0.00,0.00


In [33]:
dftgseall2 = dfse852.merge(dfse452, on=mkey)

In [34]:
#dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [35]:
dftgseall2

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,3.32,11.53,17.42,34.08,46.40,84.99,9.40,25.54,...,218.14,2.10,8.74,11.54,16.88,30.57,34.57,48.42,71.92,80.06
1,40.041039,-88.961884,3.27,11.42,17.31,33.80,46.02,84.50,9.23,25.32,...,217.79,2.10,8.68,11.43,16.82,30.42,34.18,48.17,71.65,79.53
2,40.041039,-88.878548,3.19,11.32,17.02,33.47,45.51,83.86,9.12,25.18,...,217.41,2.11,8.65,11.35,16.77,30.30,33.92,48.15,71.55,79.23
3,40.041039,-88.795219,3.08,11.12,16.58,32.85,44.62,82.89,8.90,24.78,...,216.44,2.05,8.54,11.29,16.64,30.11,33.56,47.83,71.12,78.81
4,40.041039,-88.711891,3.03,11.11,16.50,32.83,44.36,82.75,8.82,24.77,...,216.76,2.10,8.66,11.33,16.75,30.32,33.63,48.13,71.36,79.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,1.65,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,1.32,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,1.33,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,1.23,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00


In [36]:
dftgseall3 = dftgseall2.merge(dfmop50, on=mkey)
dftgseall2=0

In [37]:
dftgseall3.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_monthly.feather")

In [38]:
dftgseall3

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,3.32,11.53,17.42,34.08,46.40,84.99,9.40,25.54,...,82.00,233.76,416.28,555.58,634.25,600.83,465.64,272.86,102.54,16.52
1,40.041039,-88.961884,3.27,11.42,17.31,33.80,46.02,84.50,9.23,25.32,...,81.54,232.83,415.22,554.79,633.64,600.88,466.02,272.58,102.20,16.44
2,40.041039,-88.878548,3.19,11.32,17.02,33.47,45.51,83.86,9.12,25.18,...,80.86,232.00,414.92,554.72,633.43,601.17,466.56,272.31,101.78,16.44
3,40.041039,-88.795219,3.08,11.12,16.58,32.85,44.62,82.89,8.90,24.78,...,79.46,230.67,414.77,555.10,633.12,601.10,466.22,271.38,100.94,16.30
4,40.041039,-88.711891,3.03,11.11,16.50,32.83,44.36,82.75,8.82,24.77,...,79.45,230.60,415.72,556.49,633.40,601.68,467.61,271.97,101.15,16.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,1.65,0.00,0.00,...,0.00,0.00,0.00,10.03,67.27,52.31,1.74,0.00,0.00,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,1.32,0.00,0.00,...,0.00,0.00,0.00,7.90,59.52,45.55,1.08,0.00,0.00,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,1.33,0.00,0.00,...,0.00,0.00,0.00,9.07,64.34,49.92,1.50,0.00,0.00,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,1.23,0.00,0.00,...,0.00,0.00,0.00,9.41,65.80,51.24,1.66,0.00,0.00,0.00


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [39]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [40]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [41]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [42]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    dfpolyshape = 0
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_DJC_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_DJC_monthly.json", driver="GeoJSON")
    dftp = 0
    gdf = 0

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_DJC_monthly.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
